In [1]:
import os
print(os.getcwd())  # C:\Users\chung\Documents\04-Insight\insight\notebooks
os.chdir('../')
print(os.getcwd())  # C:\Users\chung\Documents\04-Insight\insight

from NextPick.image_search import *
import plotly.express as px
import pickle
from time import time
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline
import torch
from torch.nn import functional as F
from torchvision import transforms as trn
from annoy import AnnoyIndex
from geopy.geocoders import Nominatim

C:\Users\chung\Documents\04-Insight\insight\notebooks
C:\Users\chung\Documents\04-Insight\insight


In [2]:
input_dataset = ImageDataset('notebooks/data')
bs = 100
image_loader = torch.utils.data.DataLoader(input_dataset, batch_size=bs)
model, model_full = load_pretrained_model()

pd_files = input_dataset.get_file_df()
annoy_path = 'notebooks/annoy_idx.annoy'
if os.path.exists(annoy_path):
    annoy_idx_loaded = AnnoyIndex(512)
    annoy_idx_loaded.load(annoy_path)

C:\test\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


In [3]:
test_img = 'notebooks/ski-test-img.png'
searches = eval_test_image(test_img, model, annoy_idx_loaded, top_n=30)
df = create_df_for_map_plot(searches, pd_files)

In [4]:
# load the class label
file_name = 'places365/categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

In [5]:
def predict(img, display=False):
    with torch.no_grad():
        model_full.eval()
        data = Image.open(img,'r').convert('RGB')
        data = transform(data).unsqueeze(0)
        logit = model_full(data)
        
    h_x = F.softmax(logit, 1).data.squeeze()
    probs, idx = h_x.sort(0, True)

    # output the prediction
    if display:
        for i in range(0, 10):
            print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))
    return ([probs[i] for i in range(10)], [classes[idx[i]] for i in range(10)])

In [6]:
pred_labels = predict(test_img)

In [7]:
df.head(10)

,latitude,longitude,paths,labels,cos_diff,latlon,address,display
0,37.706137,-119.614419,/ski resort/49788543373.jpg,ski resort,0.445,"(37.706137, -119.614419)","Mariposa County, California, United States of ...",United States of America
1,46.021291,7.796773,/ski slope/49455542272.jpg,ski slope,0.510,"(46.021291, 7.796773)","Rothorn VS, Ritzengrat, Tufteren, Zermatt, Vis...",Switzerland
2,46.020918,7.797374,/ski resort/49788543528.jpg,ski resort,0.521,"(46.020918, 7.797374)","Rothorn, Ritzengrat, Tufteren, Zermatt, Visp, ...",Switzerland
3,46.021000,7.798833,/ski slope/49458494093.jpg,ski slope,0.531,"(46.021000, 7.798833)","Rothorn paradise, Ritzengrat, Tufteren, Zermat...",Switzerland
4,46.021410,7.797685,/crevasse/33853719818.jpg,crevasse,0.531,"(46.021410, 7.797685)","Rothorn VS, Ritzengrat, Tufteren, Zermatt, Vis...",Switzerland
5,46.021276,7.797085,/ski slope/49459199117.jpg,ski slope,0.536,"(46.021276, 7.797085)","Rothorn VS, Ritzengrat, Tufteren, Zermatt, Vis...",Switzerland
6,47.697429,23.799648,/ski slope/49458496608.jpg,ski slope,0.552,"(47.697429, 23.799648)","Baia Sprie, Maramureș, România",România
7,47.144196,7.062942,/ski slope/49458493663.jpg,ski slope,0.561,"(47.144196, 7.062942)","Ski Club, 10, Chemin de la Combe à Maillet, Co...",Switzerland
8,45.864552,6.881217,/snowfield/46732624584.jpg,snowfield,0.580,"(45.864552, 6.881217)","Piton Central, Chamonix-Mont-Blanc, Bonneville...",France
9,69.473329,19.891433,/chalet/49911873056.jpg,chalet,0.584,"(69.473329, 19.891433)","Lyngen, Troms og Finnmark, Norge",Norge


In [8]:
from NextPick.plotly_map import get_distances, get_top5_distance
input_latlon = (43.70011, -79.4163) #Toronto, ON, CA
df = get_distances(input_latlon, df)
df = get_top5_distance(df)

...Ascending


In [9]:
df.head()

,latitude,longitude,paths,labels,cos_diff,latlon,address,display,dist
0,37.706137,-119.614419,/ski resort/49788543373.jpg,ski resort,0.445,"(37.706137, -119.614419)","Mariposa County, California, United States of ...",United States of America,3427.0
1,47.144196,7.062942,/ski slope/49458493663.jpg,ski slope,0.561,"(47.144196, 7.062942)","Ski Club, 10, Chemin de la Combe à Maillet, Co...",Switzerland,6416.0
2,69.473329,19.891433,/chalet/49911873056.jpg,chalet,0.584,"(69.473329, 19.891433)","Lyngen, Troms og Finnmark, Norge",Norge,5877.0
3,62.966975,-151.170503,/ski slope/49458971296.jpg,ski slope,0.589,"(62.966975, -151.170503)","Basecamp, Denali, Alaska, United States of Ame...",United States of America,4916.0
4,45.682911,6.002663,/crevasse/41781791592.jpg,crevasse,0.646,"(45.682911, 6.002663)","Village Tipi, Route Nationale d'Aix Les Bains,...",France,6431.0


In [11]:
# check if these labels are in top 5 or top 10
def validation(df, labels):
    val = []
    overlap = []
    # labels is a tuple of 2 lists of probabilities and labels
    for i, row in df.iterrows():
        path = ''.join(['notebooks/data', row['paths']])
        preds = predict(path)
        val.append(preds[1])
        overlap.append(list(set.intersection(set(preds[1]), set(labels[1]))))
    df['preds'] = val
    df['overlap'] = overlap
    df['overlap_cnt'] = df['overlap'].apply(len)
    return df


df_val = validation(df, pred_labels)

In [12]:
df_val.head()

,latitude,longitude,paths,labels,cos_diff,latlon,address,display,dist,preds,overlap,overlap_cnt
0,37.706137,-119.614419,/ski resort/49788543373.jpg,ski resort,0.445,"(37.706137, -119.614419)","Mariposa County, California, United States of ...",United States of America,3427.0,"[ski_slope, snowfield, ice_shelf, glacier, ski...","[ice_floe, ski_slope, ski_resort, crevasse, gl...",9
1,47.144196,7.062942,/ski slope/49458493663.jpg,ski slope,0.561,"(47.144196, 7.062942)","Ski Club, 10, Chemin de la Combe à Maillet, Co...",Switzerland,6416.0,"[ski_slope, snowfield, glacier, mountain_snowy...","[ski_slope, ski_resort, crevasse, glacier, mou...",8
2,69.473329,19.891433,/chalet/49911873056.jpg,chalet,0.584,"(69.473329, 19.891433)","Lyngen, Troms og Finnmark, Norge",Norge,5877.0,"[mountain_snowy, ski_slope, snowfield, ice_she...","[ski_slope, ski_resort, crevasse, glacier, mou...",7
3,62.966975,-151.170503,/ski slope/49458971296.jpg,ski slope,0.589,"(62.966975, -151.170503)","Basecamp, Denali, Alaska, United States of Ame...",United States of America,4916.0,"[ski_slope, snowfield, mountain_snowy, glacier...","[ski_slope, ski_resort, crevasse, glacier, mou...",7
4,45.682911,6.002663,/crevasse/41781791592.jpg,crevasse,0.646,"(45.682911, 6.002663)","Village Tipi, Route Nationale d'Aix Les Bains,...",France,6431.0,"[mountain_snowy, glacier, ice_shelf, ski_slope...","[ice_floe, ski_slope, crevasse, glacier, mount...",8


In [ ]:
test_img1 = 'notebooks/test-img-war-mem-1.jpg'
searches1 = eval_test_image(test_img1, model, annoy_idx_loaded)
df1 = create_df_for_map_plot(searches1, pd_files)

In [ ]:
predict(test_img1)

In [ ]:
df1.head()

In [ ]:
test_img2 = 'notebooks/banff.jpg'
searches2 = eval_test_image(test_img2, model, annoy_idx_loaded)
df2 = create_df_for_map_plot(searches2, pd_files)

In [ ]:
predict(test_img2)

In [ ]:
df2.head()

In [ ]:
plot_map(searches2, pd_files)